<a href="https://colab.research.google.com/github/wooihaw/practical_ai/blob/main/three_animals_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Change directory to /content/
cd /content/

In [ ]:
# Extract dataset from Google Drive
!tar -xvf "/content/drive/My Drive/three_animals.tar.gz"

In [ ]:
# Load modules
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten

In [ ]:
# Setup training images and testing images
train_dir = 'three_animals/train'
test_dir = 'three_animals/validate/'

train_datagen = ImageDataGenerator(
                    rescale=1/255.,
                    rotation_range=30,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1/255.)

train_generator = train_datagen.flow_from_directory(
                    train_dir,
                    batch_size=32,
                    target_size=(150,150),
                    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
                    test_dir,
                    batch_size=32,
                    target_size=(150,150),
                    class_mode='categorical')

In [ ]:
# Construct CNN
model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.summary()

In [ ]:
# Compile and train model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history = model.fit(train_generator, 
                    steps_per_epoch=85, 
                    epochs=5, 
                    validation_data=test_generator,
                    validation_steps=10,
                    verbose=1)

In [ ]:
# Plot model accuracy and lose
import matplotlib.pyplot as plt

acc = history.history['acc']
loss = history.history['loss']
val_acc = history.history['val_acc']
val_loss = history.history['val_loss']

plt.plot(range(len(acc)), acc, 'b', label='accuracy')
plt.plot(range(len(val_acc)), val_acc, 'g', label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.legend(loc=0)
plt.show()

In [ ]:
# Test model on new image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
fn = 'three_animals/animal02.jpg'
img = load_img(fn, target_size=(150, 150))
x = img_to_array(img)/255.
x = np.expand_dims(x, axis=0)

classes = model.predict(x, batch_size=10)
class_indices = train_generator.class_indices
print(class_indices)
map2class = {class_indices[k]:k for k in class_indices}
print(map2class[classes.argmax()])